In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import geemap
import geetools
from datetime import datetime
import pytz

In [2]:
ee.Initialize(project='ee-jonstar')

In [3]:
#ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [19]:
tz = pytz.timezone('US/Eastern')

def toUTC(d):
    return tz.normalize(tz.localize(d)).astimezone(pytz.utc)

## CONUS Geometry

In [8]:
conus = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('2000-01-01', '2001-01-01')
geometry = conus.geometry();

In [9]:
# DMV points

# Define point near Baltimore
u_lon = -76.6122
u_lat = 39.2904
DMV_poi = ee.Geometry.Point(u_lon, u_lat)

# Point for Baltimore SAR image
lon1 = -78.08
lat1 = 39.417
pt1 = ee.Geometry.Point(lon1, lat1)

# Point for Washington SAR image
lon2 = -77.723
lat2 = 38.686
pt2 = ee.Geometry.Point(lon2, lat2)

In [44]:
# NYC points

# Define point in, NYC, works for Landsat and first SAR image
u_lon = -73.274
u_lat = 40.688
NYC_poi1 = ee.Geometry.Point(u_lon, u_lat)

# Point for second NYC SAR image
lon2 = -73.060
lat2 = 40.230
NYC_poi2 = ee.Geometry.Point(lon2, lat2)

In [5]:
# Atlanta points

# Define point slightly northeast of Atlanta, works for Sar image and first Landsat image
u_lon = -84.344
u_lat = 33.818
Atlanta_poi1 = ee.Geometry.Point(u_lon, u_lat)

# Point for second Atlanta Landsat image
lon2 = -84.465
lat2 = 33.667
Atlanta_poi2 = ee.Geometry.Point(lon2, lat2)

In [5]:
# Phoenix points

# Define point near downtown Phoenix
u_lon = -112.02
u_lat = 33.46
Phoenix_poi = ee.Geometry.Point(u_lon, u_lat)

## Landsat Initialization

In [5]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [6]:
combine_Landsat = False

In [8]:
# For filtering to multipe Landsat images

# Atlanta points
pt1_filt = ee.Filter.bounds(Atlanta_poi1)
pt2_filt = ee.Filter.bounds(Atlanta_poi2)
Landsat_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)

combine_Landsat = True

NameError: name 'Atlanta_poi1' is not defined

In [10]:
#landsat_US = landsat.filterDate('2023-01-01', '2023-12-31').filterBounds(Phoenix_poi)
#landsat_US = landsat.filterDate('2023-01-01', '2023-12-31').filterBounds(NYC_poi1)
landsat_US = landsat.filterDate('2023-01-01', '2023-12-31').filterBounds(DMV_poi)
#landsat_US = landsat.filterDate('2023-01-01', '2023-12-31').filter(Landsat_pts_filt)

In [9]:
landsat_US.first().projection()

In [10]:
print(landsat_US.size().getInfo())

20


In [12]:
lList = landsat_US.toList(15)

In [13]:
# List of days Landsat should have an image, noting missing ones (for 2024, though I am switching to 2023 data)
# January 14, 2024
# January 30, 2024 - MISSING
# February 15, 2024
# March 2, 2024 - MISSING
# March 18, 2024
# April 3, 2024 - MISSING
# April 19, 2024 - MISSING
# May 5, 2024 - MISSING
# May 21, 2024
# June 6, 2024
# June 22, 2024
# July 8, 2024
# July 24, 2024 - MISSING
# August 9, 2024 - MISSING
# August 25, 2024

ee.Image(lList.get(7)).date()

In [20]:
dt = datetime.fromtimestamp(1685893548086/1000)

In [22]:
print(dt)

# checking timezone information
print(toUTC(dt))

2023-06-04 11:45:48.086000
2023-06-04 15:45:48.086000+00:00


In [14]:
ee.Image(lList.get(7)).date()

In [15]:
# Used where two Landsat images are required, splits data into the separate images
# so that time should be equivalent among equal indexes in the two lists

#landsat_list1 = ee.ImageCollection(landsat_US.toList(19))
#landsat_list2 = ee.ImageCollection(landsat_US.toList(36, 19))

## GOES Initialization

In [6]:
# Landsat images over Baltimore regularly taken around 15:47 every 16 days
hour_filt1 = ee.Filter.calendarRange(15, 15, 'hour')
minute_filt_DMV = ee.Filter.calendarRange(45, 49, 'minute') # MUST FIGURE OUT WHEN OTHER LANDSAT IMAGES ARE TAKEN
time_filt1 = ee.Filter.And(hour_filt1, minute_filt_DMV)

hour_filt2 = ee.Filter.calendarRange(3, 3, 'hour')
time_filt2 = ee.Filter.And(hour_filt2, minute_filt_DMV)

time_filt = ee.Filter.Or(time_filt1, time_filt2)

In [7]:
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate('2023-06-01', '2023-06-30')\
                .filter(minute_filt_DMV)

In [39]:
# Time is in UTC
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate('2023-06-14T17:00', '2023-06-14T18:05')

In [8]:
gList = GOES.toList(8)

In [12]:
ee.Image(gList.get(0)).date()

In [24]:
dt = datetime.fromtimestamp(1685580377400/1000)
dt

datetime.datetime(2023, 5, 31, 20, 46, 17, 400000)

In [25]:
print(toUTC(dt))

2023-06-01 00:46:17.400000+00:00


In [20]:
aware_obj = tz.localize(dt)
 
# checking timezone information
print(aware_obj)

2023-05-31 20:46:17.400000+00:00


## Sentinel-1 SAR Initialization

In [21]:
# DMV points
pt1_filt = ee.Filter.bounds(pt1)
pt2_filt = ee.Filter.bounds(pt2)

SAR_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)

In [56]:
# NYC points
pt1_filt = ee.Filter.bounds(NYC_poi1)
pt2_filt = ee.Filter.bounds(NYC_poi2)

SAR_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)

In [22]:
global combine_SAR
sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2023-01-01', '2023-12-31').filter(SAR_pts_filt)
combine_SAR = True
#sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2023-01-01', '2023-12-31').filterBounds(Phoenix_poi)
#combine_SAR = False

In [23]:
print(sentinel.size().getInfo())

60


In [24]:
sentinel.first()

In [14]:
#slist = sentinel.toList(2)

In [15]:
#ee.Image(slist.get(1)).date()

## NLCD Initialization

In [27]:
nlcd = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER").filterDate('2022', '2024')

In [28]:
nlcd.size()

## Filtering images by time

In [22]:
#join = ee.Join.inner()
all_join = ee.Join.saveAll(matchesKey='matched_img', measureKey='time_diff', outer=True)
best_join = ee.Join.saveBest(matchKey='matched_img', measureKey='time_diff', outer=True)

#timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=518400e4, leftField='system:time_start', rightField='system:time_start')

# Difference value is equal to 6 days in milliseconds
timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=5184e5, leftField='system:time_start', rightField='system:time_start')

In [23]:
# Preprocessing function
def process_multiple_Landsat(image):
    ######################################
    # Combines Landsat images matched by time into one image
    
    match = ee.Image(image.get('matched_img'))
    median_image = ee.ImageCollection([image, match]).median()

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(image.get('system:time_start'))       
        return f.set('system:time_start', Landsat_time)

    # Add timestamp back in
    median_image = get_Landsat_time(median_image)
    
    return median_image

In [24]:
if combine_Landsat:
    all_join = ee.Join.saveAll(matchesKey='matched_img', measureKey='time_diff', outer=False)
    Landsat_near = ee.ImageCollection(best_join.apply(landsat_list1, landsat_list2, timeFiltLandsatSentinel))
    landsat_US = Landsat_near.map(process_multiple_Landsat)

In [25]:
#col_near = ee.ImageCollection(best_join.apply(GOES, landsat_US, timeFiltLandsatGOES))
LandsatSentinel_near = ee.ImageCollection(all_join.apply(landsat_US, sentinel, timeFiltLandsatSentinel))

In [23]:
landsat_US.size().getInfo()

20

In [15]:
GOES.size().getInfo()

696

In [16]:
Landsat_near.size().getInfo()

NameError: name 'Landsat_near' is not defined

In [24]:
LandsatSentinel_near.size().getInfo()

20

In [28]:
Landsat_near.first()

In [25]:
ee.List(LandsatSentinel_near.first().get('matched_img')).get(0)

In [37]:
imgs = Landsat_near.toList(3)

In [39]:
img = ee.Image(imgs.get(2))
img

In [42]:
#median = ee.Reducer.median()
#median_img = ee.Image(ee.ImageCollection(ee.List(img.get('matched_img'))).median())
match = ee.Image(img.get('matched_img'))
median_img = ee.ImageCollection([img, match]).median()

In [43]:
median_img

## Helper Functions

In [46]:
# Applies scale and offset factors for Landsat band 10 LST
def scale_and_offset_LST(image):
    return image.select('ST_B10').multiply(0.00341802).add(149)

# Applies scale and offset factors for Landsat reflectance bands
def scale_and_offset_Landsat(image):
    return image.select(['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(2.75e-05).add(-0.2)

In [47]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image):
    #reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    brightness_temps = image.select(['CMI_C13', 'CMI_C14', 'CMI_C15', 'CMI_C16']).multiply(0.039316241).add(173.14999)
    
    #return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])
    return brightness_temps

In [48]:
# Preprocessing function
def process_GOES(image):
    #####################################
    # Landsat portion
    
    # Scaling and offset
    #image1 = ee.Image(feature.get('primary'))
    #landsat_LST = scale_and_offset_LST(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    #landsat_image = landsat_image.addBands(srcImg=landsat_LST, names=['ST_B10'])\
    #                            .addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # GOES portion

    # Scaling and offset
    #image2 = ee.Image(feature.get('matched_img'))
    GOES_image = scale_and_offset_GOES(image)

    ######################################
    # Timestamp portion
    
    # Function to get time from GOES image\    
    def get_GOES_time(f):
        GOES_time = ee.Number(image.get('system:time_start'))       
        return f.set('timestamp', GOES_time)

    # Add timestamp back in
    #landsat_image = get_GOES_time(landsat_image)
    GOES_image = get_GOES_time(GOES_image)

    ##########################################
    
    #image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
    #                                                         'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return GOES_image

In [49]:
# Preprocessing function
def process_Landsat_Sentinel(feature):
    ######################################
    # Sentinel portion

    # Gather SAR image
    if combine_SAR:
        sentinel_image = ee.Image(ee.ImageCollection(ee.List(feature.get('matched_img'))).median())
    else:
        sentinel_image = ee.Image(ee.List(feature.get('matched_img')).get(0))
    
    #####################################
    # Landsat portion
    
    # Scaling and offset
    feature = ee.Image(feature)
    #image1 = ee.Image(feature.get('primary'))
    landsat_LST = scale_and_offset_LST(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    landsat_image = landsat_LST.addBands(srcImg=feature, names=['QA_PIXEL'])
                                #.addBands(srcImg=landsat_LST, names=['ST_B10'])

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(feature.get('system:time_start'))       
        return f.set('timestamp', Landsat_time)

    # Add timestamp back in
    landsat_image = get_Landsat_time(landsat_image)

    ##########################################
    
    image = landsat_image.addBands(srcImg=sentinel_image).cast({'QA_PIXEL':'double', 'angle':'double'})
    
    return image

## Produce GOES geotiffs and their respective times

In [50]:
processed = GOES.map(process_GOES)

In [51]:
first = processed.first()

In [52]:
first

In [53]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

In [54]:
times = processed.aggregate_array('timestamp')
features = ee.FeatureCollection(times.map(times_to_features))

In [59]:
times.get(0).getInfo()

1685580377400

In [27]:
datetime.fromtimestamp(1685580377400/1000)

datetime.datetime(2023, 5, 31, 20, 46, 17, 400000)

In [56]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='GOES_times_DMV',
    folder='GOES_DMV',
    fileFormat='CSV',
)
task.start()

## Produce Landsat/Sentinel geotiffs and their respective times

In [37]:
processed2 = LandsatSentinel_near.map(process_Landsat_Sentinel)

In [38]:
first = ee.Image(processed2.first())

In [39]:
first

In [40]:
times2 = processed2.aggregate_array('timestamp')
features2 = ee.FeatureCollection(times2.map(times_to_features))

In [41]:
task = ee.batch.Export.table.toDrive(
    collection=features2,
    description='Landsat_times_DMV',
    folder='Landsat_SAR_DMV',
    fileFormat='CSV',
)
task.start()

## Export stuff

In [61]:
# Export GOES images, only taking area over Baltimore and Washington
num = processed.size().getInfo()
#num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_image_{i}', fileFormat='GeoTIFF', folder='GOES_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

In [42]:
# Export Landsat/Sentinel images, only taking area over Baltimore and Washington
num = processed2.size().getInfo()
#num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{i}', fileFormat='GeoTIFF', folder='Landsat_SAR_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

In [29]:
# Export NLCD images, only taking area over Baltimore and Washington
num = nlcd.size().getInfo()
#num = 1
img_list = nlcd.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'NLCD_DMV_image_{i+2022}', fileFormat='GeoTIFF', folder='NLCD_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

In [89]:
# Export images, only taking area over NYC
num = processed.size().getInfo()
#num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_image_{i}', fileFormat='GeoTIFF', folder='GOES_NYC',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 562680, 0, -30, 4550000])
    task.start()

In [87]:
# Export images, only taking area over NYC
num = processed2.size().getInfo()
#num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{i}', fileFormat='GeoTIFF', folder='Landsat_SAR_NYC',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 562680, 0, -30, 4550000])
    task.start()

In [86]:
# Export images, only taking area over NYC
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{i}', assetId='projects/ee-jonstar/assets/NYC_test',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 562680, 0, -30, 4550000])
    task.start()

In [46]:
# Export images, only taking area over Phoenix
num = processed.size().getInfo()
#num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_image_{i}', fileFormat='GeoTIFF', folder='GOES_Phoenix',
                dimensions='3000x3000', crs='EPSG:32612', crsTransform=[30, 0, 350000, 0, -30, 3749000])
    task.start()

In [27]:
# Export images, only taking area over Phoenix
num = processed2.size().getInfo()
#num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{i}', fileFormat='GeoTIFF', folder='Landsat_SAR_Phoenix',
                dimensions='3000x3000', crs='EPSG:32612', crsTransform=[30, 0, 350000, 0, -30, 3749000])
    task.start()

In [48]:
# Export images, only taking area over Phoenix
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{i}', assetId='projects/ee-jonstar/assets/Phoenix_test',
                dimensions='3000x3000', crs='EPSG:32612', crsTransform=[30, 0, 350000, 0, -30, 3749000])
    task.start()

## Try from gee code

In [28]:
geo = sample.select('ST_B10').geometry()

In [29]:
Map = geemap.Map(center=[u_lat, u_lon], zoom=4)
Map.add_basemap("HYBRID")

Map.addLayer(sample.select('ST_B10'), {'min':250, 'max':325,
    'palette':['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}, 'LST')
Map.addLayer(sample.select('CMI_C14').clip(geo), {'min':96, 'max':342,
    'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14')

In [30]:
Map

Map(center=[39.2904, -76.6122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…